45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

始める  で      ここで
見る    は を   吾輩は ものを

In [1]:
import re

class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface # 表層形(surface)
        self.base    = base    # 基本形(base)
        self.pos     = pos     # 品詞(pos)
        self.pos1    = pos1    # 品詞細分類1(pos1)
        
    def __str__(self):
        '''オブジェクトの文字列表現'''
        return 'surface[{}]\tbase[{}]\tpos[{}]\tpos1[{}]'\
            .format(self.surface, self.base, self.pos, self.pos1)

In [2]:
class Chunk:
    def __init__(self):
        '''初期化'''
        self.morphs = []
        self.srcs   = []   # 係り元文節インデックス番号のリスト
        self.dst    = -1   # 係り先文節インデックス番号(初期値:-1, 係り先がない場合は-1のまま)
        
    def __str__(self):
        '''オブジェクトの文字列表現'''
        surface = ''
        for morph in self.morphs:
            surface += morph.surface
        return '{}\tsrcs{}\tdst[{}]'.format(surface, self.srcs, self.dst)
    
    def output_surface_wo_pct(self):
        surface = ''
#        pos_ok  = False
        for morph in self.morphs:
#            if morph.pos == pos:
#                pos_ok = True
            if morph.pos != '記号':
                surface += morph.surface
                #print(pos_ok)
#        if pos_ok == False:
#            return ''
#        else:
        return surface

In [3]:
def neko_lines():
    with open('./neko.txt.cabocha3', encoding='utf-8') as neko_cabocha:
    
        chunks = dict()     # idxをkeyにChunkを格納
        
        for line in neko_cabocha:
            if line[:3] == 'EOS':
                
                # Chunkのリストを返す
                if len(chunks) > 0:

                    # chunksをkeyでソートし、valueのみ取り出し
                    sorted_tuple = sorted(chunks.items(), key=lambda x: x[0])
                    yield list(zip(*sorted_tuple))[1]  #[1]がリストのvalue部分
                    chunks.clear()

                else:
                    yield []
                    
            # 先頭が*の行は係り受け解析結果なので、Chunkを作成
            elif line[0] == '*':

                # Chunkのインデックス番号と係り先のインデックス番号取得
                cols = re.split('\s|D', line)
                idx = int(cols[1]) # Chunkのインデックス番号
                dst = int(cols[2]) # 係り先文節インデックス番号

                # Chunkを生成（なければ）し、係り先のインデックス番号セット
                if idx not in chunks:
                    chunks[idx] = Chunk()
                chunks[idx].dst = dst

                # 係り先のChunkを生成（なければ）し、係り元インデックス番号追加
                if dst != -1:
                    if dst not in chunks:
                        chunks[dst] = Chunk()
                    chunks[dst].srcs.append(idx) # 係り元は複数あるのでappend
                    
            else:
                
                #タブとカンマで分割
                cols = re.split('\t|,', line)

                chunks[idx].morphs.append(Morph(
                        cols[0],    # 表層形(surface)
                        cols[7],    # 基本形(base)
                        cols[1],    # 品詞(pos)
                        cols[2]     # 品詞細分類1(pos1)
                    ))
        #print('stop')
        #raise StopIteration

In [4]:
def output_verb_with_joshi(chunk):
    # 動詞のみループ
    for verb in chunk.morphs:
        if verb.pos == '動詞':
            joshi_list = []

            # 係り元ループ
            for src in chunk.srcs:
                
                last_joshi = ''
                sentence = ''

                # 係り元の助詞のみループ
                for source in chunks[src].morphs:
                    sentence = sentence + source.surface
                    if source.pos == '助詞':
                        last_joshi = source.base  #退避
                
                # 助詞があったら追加
                if last_joshi != '':
                    joshi_list.append([last_joshi, chunks[src].output_surface_wo_pct()])

            # ファイル出力
            if len(joshi_list) > 0:
                joshi_list.sort()
                out_file.write('{}\t{} {}\n'.format(verb.base, 
                                                    ' '.join([line[0] for line in joshi_list]),    #助詞
                                                    ' '.join([line[1] for line in joshi_list])) )  #文節
            
            # 動詞は1形態素につき最左のみなので１つ目でbreak
            break

In [5]:
# 結果ファイル作成
with open('./046_result.txt', mode='w') as out_file:

    # 1文ずつリスト作成
    for chunks in neko_lines():

        # 1文
        for chunk in chunks:
            # 動詞の各パターンを抽出して出力
            output_verb_with_joshi(chunk)